## Cumulative xGSA Charts at All-Star Break

Top 10 Goalies

- Jacob Markstrom
- Connor Hellebuyck
- Thatcher Demko
- Connor Ingram
- Adin Hill
- Jeremy Swayman
- Tristan Jarry
- Jordan Binnington
- Joey Daccord
- John Gibson

Bottom 10 Goalies on Nov 23rd

- Stuart Skinner
- Vitek Vanecek
- Filip Gustavsson
- Marc-Andre Fleury
- Samuel Ersson
- Dan Vladar	
- Anton Forsberg
- Alexandar Georgiev		
- Ilya Samsonov
- Philipp Grubauer


In [295]:
import pandas as pd
import glob
import re
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import hex_to_rgb
pd.set_option('mode.chained_assignment', None)

In [296]:
# Create dict of dataframes from csv
goaliePaths = [file for file in glob.glob('goalie-csv/*.csv',)]

data = {re.split("_", file, 5)[4]: pd.read_csv(file) for file in goaliePaths}
for goalie in data.values():
    goalie.drop(goalie.tail(1).index, inplace=True)

data["skinner"].head()

,Player,Date,Game_ID,Season,Team,Opponent,Is_Home,Position,TOI,TOI%,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAx
0,Stuart Skinner,2023-10-11,2023020009,20232024,EDM,VAN,0,G,32.50,54.17,3.96,17.26,19.40,1.36,77.03,79.57,93.01,-13.44,-2.61
1,Stuart Skinner,2023-10-14,2023020029,20232024,EDM,VAN,1,G,57.63,96.06,4.19,16.48,24.94,2.41,74.58,83.21,90.35,-7.14,-1.78
2,Stuart Skinner,2023-10-21,2023020077,20232024,EDM,WPG,1,G,63.73,99.77,2.98,26.40,35.33,2.70,88.71,91.56,92.36,-0.80,-0.28
3,Stuart Skinner,2023-10-26,2023020110,20232024,EDM,NYR,1,G,60.00,100.00,3.22,35.18,50.02,2.88,90.85,93.56,94.25,-0.69,-0.34
4,Stuart Skinner,2023-10-29,2023020129,20232024,EDM,CGY,1,G,59.75,99.58,2.08,25.38,44.84,3.07,91.80,95.36,93.16,2.20,0.99


In [297]:
data["skinnerStart"] = data["skinner"].head(13)
data["skinnerNov23"] = data["skinner"].tail(21)

In [298]:
# Calculate cummulative GSAx and add Total GSAx by game as seperate column for each goalie
for goalie in data.values():
    totalGSAx = 0
    GSAxList = []
    for game in goalie["GSAx"]:
        totalGSAx += game
        GSAxList.append(totalGSAx)
    goalie["Total GSAx"] = GSAxList


In [299]:
fig = go.Figure()
for goalie in data.items():

    bgColor = px.colors.qualitative.Plotly[list(data.keys()).index(goalie[0]) % 10] 
    r, g, b = hex_to_rgb(bgColor)
    if (r*0.2126 + g*0.7152 + b*0.0722) > 170: 
        textColor = "#000000"
    else: 
         textColor = "#ffffff"

    fig.add_trace(go.Scatter(y=goalie[1]['Total GSAx'], name=goalie[0], mode='lines'))
    fig.add_annotation(x=len(goalie[1]) - 1, y=goalie[1]['Total GSAx'].iloc[-1], 
            text=goalie[0], showarrow=False, 
            font=dict(size=11, color=textColor),
            bgcolor=bgColor, opacity=0.85, 
            borderpad=2.5
            )
fig.update_layout(width=1000, height=700)
fig.show()
